In [62]:
import gzip
import pandas as pd
import os
import glob
import json

In [15]:
genes = {}
with open('genes.txt', 'r') as file:
    content = file.readlines()
    for line in content:
        terms = line.split('\t')
        genes[terms[1].strip()] = terms[0]

In [56]:
cases = []
for i in glob.glob('datasets/*'):
    filename = glob.glob(f'{i}/*.gz')
    case = {}
    case['filename'] = filename[0].split('\\')[-1]
    with gzip.open(filename[0], 'rt') as file:
        content = file.readlines()
        for line in content:
            terms = line.split('\t')
            gene_key = terms[0].split('.')[0]
            if gene_key in genes.keys():
                case[genes[gene_key]] = float(terms[1].strip())
    cases.append(case)

In [57]:
df = pd.DataFrame.from_records(cases)

In [59]:
case_id_df = pd.read_json('file-case_map.json')

In [67]:
case_id_map = {}
with open('file-case_map.json', "r") as read_file:
    data = json.load(read_file)
    for line in data:
        case_id = line['cases'][0]['case_id']
        file_name = line['file_name']
        case_id_map[case_id] = file_name

In [81]:
case_id_map_df = pd.DataFrame.from_dict(case_id_map, orient='index')
case_id_map_df.reset_index(inplace=True)
case_id_map_df.columns = ['case_id','filename']

In [88]:
df = pd.merge(df, case_id_map_df, how='left', on='filename')

In [89]:
case_id_2_df = pd.read_csv('uuid-case_map.txt', sep='\t', names = ['case_id','uuid'])

In [90]:
df = pd.merge(df, case_id_2_df, how='left', on='case_id')

In [91]:
df.head()

,filename,BRCA2,RAF1,HRAS,PIK3CA,CHEK2,MYC,BRCA1,PTEN,STK11,PALB2,TP53,CDH1,ERBB2,case_id,uuid
0,a5dc521e-bee4-489c-8679-d4b90a327d33.FPKM.txt.gz,0.405744,20.514745,5.346740,4.531447,6.498114,61.566397,6.068113,16.156906,2.219489,5.669937,20.498861,264.985569,39.619147,ba89cb4f-cd63-47b1-9550-76fdaff89f2e,TCGA-A2-A25E
1,d229a59b-4f6c-4714-8dd5-1c714a764b22.FPKM.txt.gz,0.915719,16.426665,14.341597,4.505233,3.164234,12.457714,1.323357,13.851366,5.810507,4.379566,15.941618,9.232411,25.470280,959ff069-8a49-4c9b-85c2-5291cac0acff,TCGA-C8-A3M7
2,ee05eaa6-eaca-4438-ac1f-4259f525e2a1.FPKM.txt.gz,1.947190,11.657686,19.473508,3.775207,2.373475,4.430400,2.375667,8.468728,3.759698,3.786047,7.726373,111.562275,103.689169,7f2a63e6-64d5-4be1-9814-e8416fc9e688,TCGA-D8-A1JG
3,4bf2bea7-00ef-485c-b9e5-035f0ea62b1d.FPKM.txt.gz,1.383030,14.515607,16.124391,4.802638,1.741451,10.286806,1.599496,9.812499,8.161780,3.882405,22.701470,104.415741,25.581075,86b7b206-8e69-432d-acc9-bccce710955e,TCGA-AO-A0JA
4,31e5c917-1866-4156-9eaa-84a72c310edc.FPKM.txt.gz,0.283229,10.515834,19.042857,3.186440,1.960279,58.891972,1.756502,9.513710,8.948217,2.437246,13.688792,85.650483,19.833254,a53dd490-e0ba-419a-90c6-b905e241a8e5,TCGA-A2-A4S0


In [115]:
duplicates = pd.read_csv('RNAseq files.csv', index_col = 0, usecols=['Case-ID', 'Tumour 1'])

In [117]:
duplicates = duplicates.to_dict()['Tumour 1']

In [120]:
for i in range(len(df)):
    if df.loc[i, 'case_id'] in duplicates.keys():
        if df.loc[i, 'case_id'] != duplicates[df.loc[i, 'case_id']]:
            df.loc[i, 'case_id'] = None

In [121]:
df['case_id'].isnull().sum()

95

In [122]:
len(df)

1057

In [123]:
df.dropna(inplace=True)

In [124]:
len(df)

962

In [126]:
df.to_csv('prognostic_genes.csv')